In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf

In [ ]:
train_start_date = '2016-01-01'
train_end_date = '2022-12-31'
predict_start_date = '2023-01-01'
predict_end_date = '2023-12-31'

btc = yf.download('BTC-USD', start=train_start_date, end=predict_end_date)
print(btc.head())

In [ ]:
# Wyodrębnij dane do treningu i testowania na podstawie dat
date = pd.to_datetime(predict_start_date)
new_date = date - pd.Timedelta(days=100)
train_data = btc.loc[train_start_date:train_end_date]['Close'].values.reshape(-1, 1)
test_data = btc.loc[new_date.strftime('%Y-%m-%d'):predict_end_date]['Close'].values.reshape(-1, 1)
print(len(train_data), len(test_data))

print(btc.loc[predict_start_date:predict_end_date])

In [198]:
# 3. Normalizacja danych
scaler = MinMaxScaler(feature_range=(0, 1))
print(type(scaler))
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)
print(len(train_data_scaled), len(test_data_scaled))

<class 'sklearn.preprocessing._data.MinMaxScaler'>
2557 464


In [ ]:
# 4. Podział danych na zestawy treningowe i testowe
# train_size = int(len(btc_close_scaled) * 0.8)  # 80% danych do treningu
# train_data, test_data = btc_close_scaled[0:train_size], btc_close_scaled[train_size:len(btc_close_scaled)]
# print(train_size)
# print(len(btc_close_scaled))


In [ ]:
# 5. Przygotowanie danych do LSTM
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 100
X_train, y_train = create_dataset(train_data_scaled, time_step)
X_test, y_test = create_dataset(test_data_scaled, time_step)


In [ ]:
# 6. Konwersja danych do formatu akceptowanego przez LSTM (samples, time_steps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
print(len(X_train), len(X_test))

In [ ]:
# 7. Tworzenie modelu LSTM
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(100, return_sequences=False),
    tf.keras.layers.Dense(25),
    tf.keras.layers.Dense(1)
])
model.summary()

In [ ]:
# Kompilacja modelu
model.compile(optimizer='adam', loss='mean_squared_error')

# 8. Trenowanie modelu
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=64, verbose=1)

In [ ]:
# 9. Predykcja na danych testowych
predictions = model.predict(X_test)
print(predictions[:10])

In [ ]:
# 10. Odwrotna transformacja dla porównania z rzeczywistymi danymi
predictions = scaler.inverse_transform(predictions.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
print(y_test[:10])

In [ ]:
print(len(predictions))
print(len(X_test))

In [ ]:
# Wizualizacja prognozy i rzeczywistych danych dla roku 2023
plt.figure(figsize=(10, 5))

# Daty testowe (dla prognoz)
test_dates = btc.loc[new_date.strftime('%Y-%m-%d'):predict_end_date].index[time_step + 1:len(predictions) + time_step + 1]
print(len(test_dates))
# Wykres przewidywań
plt.plot(test_dates, predictions, label='Predicted', color='red')

# Wyświetlanie rzeczywistych cen dla tego okresu
actual_prices = btc['Close'].loc[predict_start_date:predict_end_date]
actual_dates = actual_prices.index
print(len(actual_dates))
plt.plot(actual_dates, actual_prices, label='Actual', color='blue')

plt.title(f'Bitcoin Price Prediction ({predict_start_date} - {predict_end_date})')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.xticks(rotation=45)
plt.xlim([predict_start_date, predict_end_date])  # Ustawiamy zakres osi X
plt.show()

print(predictions)
